# demo_02_segment_glas_patches

A demonstration of running the GlaS set through HistoSegNet and evaluating the results qualitatively and quantitatively.

## Setup

In [ ]:
%matplotlib inline
import hsn_v1
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from hsn_v1.adp import Atlas
from hsn_v1.utilities import *
from hsn_v1.histonet import HistoNet
from hsn_v1.gradcam import GradCAM
from hsn_v1.densecrf import DenseCRF

## Initialize HistoSegNet

* `input_name` $\in$ {'01_tuning_patch', '02_glas_full'}: the segmentation set to be solved
* `input_size` $\in$ [int > 0, int > 0]: the resized size of the input image
* `input_mode` $\in$ {'patch', 'wsi'}: the type of input image, either patch ('patch') or slide ('wsi'); only patches supported in public code release
* `down_fac` $\in$ 0 $\leq$ float $\leq$ 1: the scalar by which to downsample the input image to ensure equivalent pixel resolution (1.2143 um/px / input pixel resolution)
* `batch_size` $\in$ int > 0: batch size of input images
* `htt_mode` $\in$ {'both', 'morph', 'func', 'glas'}: the type of classes to segment from the images: both morphological and functional types ('both'), only morphological types ('morph'), only functional types ('func'), and glandular/non-glandular ('glas')
* `gt_mode` $\in$ {'on', 'off'}: whether to assess the segmentation against the ground-truth annotations, either 'on' or 'off'
* `run_level` $\in$ {1, 2, 3}: the final stage to run in HistoSegNet: the first stage/CNN confidence scores (1), the third stage/modified Grad-CAMs (2), or the fourth stage/dense CRF segmentation masks (3)
* `save_types` $\in$ [{0, 1}, {0, 1}, {0, 1}, {0, 1}]: the types of files to save for debugging, as a list of four binary values (0/1)
  1. HTT confidence scores: save (1), do not save (0)
  2. Continuous Grad-CAMs: save (1), do not save (0)
  3. Discrete segmentation masks: save (1), do not save (0)
  4. Summary images: save (1), do not save (0)
* `verbosity` $\in$ {'NORMAL', 'QUIET'}: the verbosity of debug messages, either on ('NORMAL') or off ('QUIET')

In [ ]:
IN_PX_RESOL = 0.620 # um/px, for GlaS images
OUT_PX_RESOL = 0.25 * 1088 / 224    # 1.21428571429 um/px, for ADP images (0.25 um/px at 224px)
DOWNSAMPLE_FACTOR = OUT_PX_RESOL / IN_PX_RESOL

hsn = hsn_v1.HistoSegNetV1(params={'input_name': '02_glas_full', 'input_size': [224, 224], 
                                   'input_mode': 'patch', 'down_fac': DOWNSAMPLE_FACTOR, 
                                   'batch_size': 1, 'htt_mode': 'glas', 'gt_mode': 'on', 
                                   'run_level': 3, 'save_types': [0, 0, 1, 0], 
                                   'verbosity': 'QUIET'})

## Load images

Load images from file

In [ ]:
def filter_train(file):
    if 'train' in file:
        return False
    else:
        return True

In [ ]:
hsn.find_img()
hsn.input_files_all

In [ ]:
test_files = list(filter(filter_train, hsn.input_files_all))
hsn.input_files_all = test_files
len(hsn.input_files_all)

Obtain logarithmic inverse class frequencies

In [ ]:
hsn.analyze_img()

Display logarithmic inverse class frequencies for the glandular/non-glandular types

In [ ]:
df = pd.DataFrame(data=hsn.httclass_loginvfreq[0], index=hsn.httclass_valid_classes[0], columns=np.array([hsn.htt_classes[0]]))
df

## Load CNN (ADP pre trained)

In [ ]:
# hsn.load_histonet(params={'model_name': 'histonet_X1.7_clrdecay_5'})
# hsn.load_histonet(params={'model_name': 'histonet_glas'})
# hsn.load_histonet(params={'model_name': 'histonet_glas_ft'})
# hsn.load_histonet(params={'model_name': 'histonet_glas_holdout'})
hsn.load_histonet(params={'model_name': 'histonet_glas_holdout_ft'})

print(hsn.hn.model.summary())

Print the loaded class score thresholds

In [ ]:
df = pd.DataFrame(data=hsn.hn.thresholds[0], index=hsn.atlas.level5)
df

## Run all batches

In [ ]:
confscores = np.zeros((len(hsn.input_files_all), len(hsn.hn.class_names)))
iou = {}
fiou = {}
miou = {}
dice = {}
mdice = {}
num_batches = (len(hsn.input_files_all) + hsn.batch_size - 1) // hsn.batch_size

for iter_batch in tqdm(range(num_batches)):
    start = iter_batch * hsn.batch_size
    end = min((iter_batch + 1) * hsn.batch_size, len(hsn.input_files_all))
    hsn.input_files_batch = hsn.input_files_all[start:end]
    # Normalize image batch
    hsn.load_norm_imgs()
    # Load ground-truth annotations
    hsn.load_gt()
    # Segment image
    hsn.segment_img()
    # Evaluate segmentation
    iou['GradCAM'], fiou['GradCAM'], miou['GradCAM'], dice['GradCAM'], mdice['GradCAM'] = hsn.eval_segmentation(hsn.intersect_counts['GradCAM'], hsn.union_counts['GradCAM'],
                                                                             hsn.confusion_matrix['GradCAM'], hsn.gt_counts['GradCAM'],
                                                                             httclass_pred_segmasks=hsn.ablative_segmasks['GradCAM'], tag_name='GradCAM')
    iou['Adjust'], fiou['Adjust'], miou['Adjust'], dice['Adjust'], mdice['Adjust'] = hsn.eval_segmentation(hsn.intersect_counts['Adjust'], hsn.union_counts['Adjust'],
                                                                          hsn.confusion_matrix['Adjust'], hsn.gt_counts['Adjust'],
                                                                          httclass_pred_segmasks=hsn.ablative_segmasks['Adjust'], tag_name='Adjust')
    iou['CRF'], fiou['CRF'], miou['CRF'], dice['CRF'], mdice['CRF'] = hsn.eval_segmentation(hsn.intersect_counts['CRF'], hsn.union_counts['CRF'],
                                                                 hsn.confusion_matrix['CRF'], hsn.gt_counts['CRF'],
                                                                 httclass_pred_segmasks=hsn.ablative_segmasks['CRF'], tag_name='CRF')

In [ ]:
dice

In [ ]:
mdice

## Qualitative performance

### Original images and Ground-truth annotations

Display original image, morphological ground truth, and functional ground truth annotations

In [ ]:
plt.figure
plt.subplot(121)
plt.imshow(hsn.input_images[0].astype('uint8'))
plt.title('Original image')
plt.subplot(122)
plt.imshow(hsn.httclass_gt_segmasks[0][0].astype('uint8'))
plt.title('Glandular/Non-glandular\n ground truth')

### Predicted segmentations, stage-by-stage

Display the predicted segmentations after Stages 2, 3, and 4 for a sample image, for the glandular/non-glandular types

In [ ]:
# plt.figure
# plt.subplot(131)
# plt.imshow(hsn.ablative_segmasks['GradCAM'][0][0])
# plt.title('(2) Pixel-level\n Segmentation\n(glandular/non-glandular)')
# plt.subplot(132)
# plt.imshow(hsn.ablative_segmasks['Adjust'][0][0])
# plt.title('(3) Inter-HTT\n Adjustments\n(glandular/non-glandular)')
# plt.subplot(133)
plt.imshow(hsn.ablative_segmasks['CRF'][0][0])
plt.title('(4) Segmentation\n Post-Processing\n(glandular/non-glandular)')

## Quantitative performance

### Glandular/Non-glandular types

Class IoU

In [ ]:
df = pd.DataFrame(data=iou['CRF'][0], index=hsn.httclass_valid_classes[0], columns=np.array(['IoU']))
df.plot.bar(y='IoU')

Frequency-weighted IoU

In [ ]:
fiou['CRF'][0]

Mean IoU

In [ ]:
miou['CRF'][0]